In [1]:

import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [2]:

red = pd.read_csv("Red.csv")
white = pd.read_csv("White.csv")
rose = pd.read_csv("Rose.csv")
sparkling = pd.read_csv("Sparkling.csv")
red["WineStyle"] = "Red"
white["WineStyle"] = "White"
rose["WineStyle"] = "Rose"
sparkling["WineStyle"] = "Sparkling"
df = pd.concat([red, white, rose, sparkling], ignore_index=True)
df = df[df["Year"] != "N.V."].copy()
df["Year"] = df["Year"].astype(int)
df["Age"] = 2025 - df["Year"]
df["Variety"] = "Unknown"
df["PricePerAge"] = df["Price"] / (df["Age"] + 1)
df["IsOld"] = df["Age"] >= 10
df["LogPrice"] = np.log1p(df["Price"])
df["PriceAgeInteraction"] = df["PricePerAge"] * df["LogPrice"]
df["LogLogPrice"] = np.log1p(df["LogPrice"])
global_mean = df["Rating"].mean()
df["SmoothedRating"] = (df["Rating"] * df["NumberOfRatings"] + global_mean * 5) / (df["NumberOfRatings"] + 5)
def reliability_group(n):
    if n < 35: return "Low"
    elif n < 838: return "Medium"
    else: return "High"
df["ReliabilityGroup"] = df["NumberOfRatings"].apply(reliability_group)
df["Country_Rank"] = df["Country"].map(df.groupby("Country")["SmoothedRating"].mean().rank(ascending=False, method="min").astype(int))
df["Region_Rank"] = df["Region"].map(df.groupby("Region")["SmoothedRating"].mean().rank(ascending=False, method="min").astype(int))
winery_mean = df.groupby("Winery")["SmoothedRating"].mean()
df["Winery_Score"] = df["Winery"].map((winery_mean.max() - winery_mean) / (winery_mean.max() - 1))
df["Variety_Freq"] = df["Variety"].map(df["Variety"].value_counts(normalize=True))
df["LogNumberOfRatings"] = np.log1p(df["NumberOfRatings"])
df["RatingSupportScore"] = df["NumberOfRatings"] / (df["Age"] + 1)


In [3]:
from sklearn.preprocessing import OrdinalEncoder

features = ["Country", "Region", "Winery", "Price", "WineStyle", "Variety", "Age",
            "NumberOfRatings", "PricePerAge", "IsOld", "LogPrice", "PriceAgeInteraction",
            "LogLogPrice", "Country_Rank", "Region_Rank", "Winery_Score", "Variety_Freq",
            "LogNumberOfRatings", "RatingSupportScore"]
target = "SmoothedRating"

# Sao chép và mã hóa
df_encoded = df.copy()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# Tìm các cột object/category và mã hóa toàn bộ
object_cols = [col for col in features if df_encoded[col].dtype == "object" or str(df_encoded[col].dtype) == "category"]
df_encoded[object_cols] = encoder.fit_transform(df_encoded[object_cols])

# Tách theo nhóm
low_df = df_encoded[df_encoded["ReliabilityGroup"] == "Low"].dropna(subset=features + [target])
mid_df = df_encoded[df_encoded["ReliabilityGroup"] == "Medium"].dropna(subset=features + [target])
high_df = df_encoded[df_encoded["ReliabilityGroup"] == "High"].dropna(subset=features + [target])

# Train/test split
from sklearn.model_selection import train_test_split
low_train, low_test = train_test_split(low_df, test_size=0.2, random_state=42)
mid_train, mid_test = train_test_split(mid_df, test_size=0.2, random_state=42)
high_train, high_test = train_test_split(high_df, test_size=0.2, random_state=42)

# Blend Mid vào Low & High
mid_sample_for_low = mid_train.sample(frac=0.5, random_state=1)
mid_sample_for_high = mid_train.sample(frac=0.4, random_state=2)
low_train_blended = pd.concat([low_train, mid_sample_for_low], ignore_index=True)
high_train_blended = pd.concat([high_train, mid_sample_for_high], ignore_index=True)


In [9]:
import os, joblib
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mae, rmse, r2, mape

results = {}

for name, train_df, test_df in [
    ("Low", low_train_blended, low_test),
    ("Medium", mid_train, mid_test),
    ("High", high_train_blended, high_test)
]:
    X_train, y_train = train_df[features], train_df[target]
    X_test, y_test = test_df[features], test_df[target]

    model = HistGradientBoostingRegressor(
        max_iter=300,
        learning_rate=0.03,
        max_depth=4,
        l2_regularization=1.0,
        early_stopping=True,
        validation_fraction=0.2
    )

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    results[name] = {
        "train": evaluate(y_train, y_train_pred),
        "test": evaluate(y_test, y_test_pred)
    }
    
    joblib.dump(model, f"models/HistGB_{name}.pkl")


In [5]:

def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mae, rmse, r2, mape

results = {}
for name, train_df, test_df in [("Low", low_train_blended, low_test), ("Medium", mid_train, mid_test), ("High", high_train_blended, high_test)]:
    X_train, y_train = train_df[features], train_df[target]
    X_test, y_test = test_df[features], test_df[target]
    model = HistGradientBoostingRegressor(max_iter=300, learning_rate=0.03, max_depth=4)
    model.fit(X_train, y_train)
    results[name] = {
        "train": evaluate(y_train, model.predict(X_train)),
        "test": evaluate(y_test, model.predict(X_test))
    }


In [6]:
print("📊 Kết quả trên tập TEST:")
for group in ["Low", "Medium", "High"]:
    mae, rmse, r2, mape = results[group]["test"]
    print(f"{group:<7} Test  — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

print("\n🧪 Kết quả trên tập TRAIN:")
for group in ["Low", "Medium", "High"]:
    mae, rmse, r2, mape = results[group]["train"]
    print(f"{group:<7} Train — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")


📊 Kết quả trên tập TEST:
Low     Test  — MAE: 0.1198, RMSE: 0.1500, R²: 0.6935, MAPE: 3.16%
Medium  Test  — MAE: 0.1148, RMSE: 0.1477, R²: 0.7182, MAPE: 2.99%
High    Test  — MAE: 0.1064, RMSE: 0.1375, R²: 0.7721, MAPE: 2.69%

🧪 Kết quả trên tập TRAIN:
Low     Train — MAE: 0.1074, RMSE: 0.1397, R²: 0.7521, MAPE: 2.82%
Medium  Train — MAE: 0.1059, RMSE: 0.1373, R²: 0.7630, MAPE: 2.77%
High    Train — MAE: 0.0996, RMSE: 0.1299, R²: 0.7948, MAPE: 2.60%


In [7]:

import pandas as pd
from IPython.display import display

df_eval = pd.DataFrame({
    "Group": ["Low", "Medium", "High"],
    "MAE_Train": [results[g]["train"][0] for g in ["Low", "Medium", "High"]],
    "MAE_Test":  [results[g]["test"][0]  for g in ["Low", "Medium", "High"]],
    "R2_Train":  [results[g]["train"][2] for g in ["Low", "Medium", "High"]],
    "R2_Test":   [results[g]["test"][2]  for g in ["Low", "Medium", "High"]]
})
df_eval["ΔMAE"] = df_eval["MAE_Test"] - df_eval["MAE_Train"]
df_eval["ΔR2"] = df_eval["R2_Test"] - df_eval["R2_Train"]

print("📉 So sánh mức độ overfit:")
display(df_eval[["Group", "MAE_Train", "MAE_Test", "ΔMAE", "R2_Train", "R2_Test", "ΔR2"]])


📉 So sánh mức độ overfit:


Group  MAE_Train  MAE_Test      ΔMAE  R2_Train   R2_Test       ΔR2
0     Low   0.107440  0.119831  0.012391  0.752052  0.693477 -0.058575
1  Medium   0.105889  0.114754  0.008865  0.763022  0.718246 -0.044776
2    High   0.099592  0.106382  0.006791  0.794794  0.772089 -0.022705

In [8]:
import os, joblib
os.makedirs("models", exist_ok=True)
joblib.dump(model, f"models/HistGB_{name}.pkl")


['models/HistGB_High.pkl']